# Simulate UT Softball Lineups using Monte Carlo
***

In [1]:
import random
import itertools

In [48]:
# UT Softball
players = [
    {"name": "Ashton Maloney", "BB":0.08, "1B":0.31, "2B":0.04, "3B":0.01, "HR":0.0},
    {"name": "Kayden Henry", "BB":0.09, "1B":0.29, "2B":0.06, "3B":0.02, "HR":0.01},
    {"name": "Mia Scott", "BB":0.13, "1B":0.24, "2B":0.07, "3B":0.01, "HR":0.04},
    {"name": "Reese Atwood", "BB":0.18, "1B":0.17, "2B":0.06, "3B":0.02, "HR":0.09},
    {"name": "Katie Stewart", "BB":0.10, "1B":0.17, "2B":0.05, "3B":0.0, "HR":0.07},
    {"name": "Joley Mitchell", "BB":0.18, "1B":0.17, "2B":0.05, "3B":0.01, "HR":0.08},
    {"name": "Victoria Hunter", "BB":0.15, "1B":0.07, "2B":0.04, "3B":0.0, "HR":0.06},
    {"name": "Vanessa Quiroga", "BB":0.06, "1B":0.13, "2B":0.02, "3B":0.0, "HR":0.07},
    {"name": "Leighann Goode", "BB":0.08, "1B":0.15, "2B":0.04, "3B":0.01, "HR":0.04},
]

In [46]:
# calculate stats from webside

# # AB + SF + SH + HBP + BB
# plate_appearances = 136 + 5 + 3 + 13 + 2
# # 1B = H - (2B+3B+HR)
# single = 38 - (6+2+6)
# stat = 6
# stat / plate_appearances


0.03773584905660377

In [49]:
# add "out" probability
for p in players:
    p['OUT'] = 1.0 - (p['BB'] + p['1B'] + p['2B'] + p['3B'] + p['HR'])

players

[{'name': 'Ashton Maloney',
  'BB': 0.08,
  '1B': 0.31,
  '2B': 0.04,
  '3B': 0.01,
  'HR': 0.0,
  'OUT': 0.56},
 {'name': 'Kayden Henry',
  'BB': 0.09,
  '1B': 0.29,
  '2B': 0.06,
  '3B': 0.02,
  'HR': 0.01,
  'OUT': 0.53},
 {'name': 'Mia Scott',
  'BB': 0.13,
  '1B': 0.24,
  '2B': 0.07,
  '3B': 0.01,
  'HR': 0.04,
  'OUT': 0.51},
 {'name': 'Reese Atwood',
  'BB': 0.18,
  '1B': 0.17,
  '2B': 0.06,
  '3B': 0.02,
  'HR': 0.09,
  'OUT': 0.48},
 {'name': 'Katie Stewart',
  'BB': 0.1,
  '1B': 0.17,
  '2B': 0.05,
  '3B': 0.0,
  'HR': 0.07,
  'OUT': 0.61},
 {'name': 'Joley Mitchell',
  'BB': 0.18,
  '1B': 0.17,
  '2B': 0.05,
  '3B': 0.01,
  'HR': 0.08,
  'OUT': 0.51},
 {'name': 'Victoria Hunter',
  'BB': 0.15,
  '1B': 0.07,
  '2B': 0.04,
  '3B': 0.0,
  'HR': 0.06,
  'OUT': 0.6799999999999999},
 {'name': 'Vanessa Quiroga',
  'BB': 0.06,
  '1B': 0.13,
  '2B': 0.02,
  '3B': 0.0,
  'HR': 0.07,
  'OUT': 0.72},
 {'name': 'Leighann Goode',
  'BB': 0.08,
  '1B': 0.15,
  '2B': 0.04,
  '3B': 0.01,
  '

In [50]:
# simulate one plate appearance
def simulate_pa(player):
    random_prob = random.random()
    thresholds = [
        ('BB', player['BB']),
        ('1B', player['1B']),
        ('2B', player['2B']),
        ('3B', player['3B']),
        ('HR', player['HR']),
    ]
    cumulative = 0
    for outcome, prob in thresholds:
        cumulative += prob
        if random_prob < cumulative:
            return outcome
    return "OUT"

For example if a random value generated is 0.28 and we have the following probailities:
- walk = 10%
- single = 20%
- double = 5%
- triple = 1%
- homerun = 6%
- out = 58%

Then we have the following thresholds:

- walk (0 -> 0.1)
- 1B (0.1 -> 0.3)
- 2B (0.3 -> 0.35)
- 3B (0.35 -> 0.36)
- HR (0.36-> 0.42)
- OUT (0.42 - 1.0)
Based on these thresholds and the random value generated we assume the event that occured was a single


In [51]:
# simulate one inning
def simulate_inning(lineup, batter_index):
    outs = 0
    runs = 0
    bases = [False, False, False] # 1B, 2B, 3B

    while outs < 3:
        player = lineup[batter_index % len(lineup)]
        result = simulate_pa(player)

        if result == "OUT":
            outs += 1
        elif result == "BB" or result == "1B":
            # walk or single so base runners advance one
            if bases[2]:
                runs += 1
                bases[2] = False
            if bases[1]:
                bases[2] = True
                bases[1] = False
            if bases[0]:
                bases[1] = True
            bases[0] = True
        elif result == "2B":
            # Double so base runners advance 2
            if bases[2]:
                runs += 1
                bases[2] = False
            if bases[1]:
                runs += 1
                bases[1] = False
            if bases[0]:
                bases[2] = True
                bases[0] = False
            bases[1] = True
        elif result == "3B":
            # triple so base runners advance 3
            runs += sum(bases)
            bases = [False, False, False]
            bases[2] = True
        elif result == "HR":
            # homerun so all runnes and batter scores
            runs += 1 + sum(bases)
            bases = [False, False, False]
        batter_index += 1
    return runs, batter_index

In [52]:
# simulate game (7 innings)
def simulate_game(lineup):
    runs = 0
    batter_index = 0
    for inning in range(1,8):
        inning_runs, batter_index = simulate_inning(lineup, batter_index)
        runs += inning_runs
    return runs

In [53]:
# simulate multiple games for same lineup
def average_runs(lineup, num_games = 1000):
    total_runs = sum(simulate_game(lineup) for _ in range (num_games))
    return total_runs / num_games

In [54]:
# randomly find best lineup
def find_best_lineup(players, num_lineups=1000):
    best_avg = 0
    best_lineup = None
    for i in range(num_lineups):
        lineup = random.sample(players, len(players))
        avg = average_runs(lineup, num_games=200)
        if avg > best_avg:
            best_avg = avg
            best_lineup = lineup
    return best_lineup, best_avg

In [68]:
# run optimizer
best_lineup, best_runs = find_best_lineup(players, num_lineups=500)

In [69]:
print('Best Line Up Order:')
for i, player in enumerate(best_lineup, start=1):
    print(f"{i}: {player['name']}")
print(f'Average runs per game = {best_runs:.2f}')

Best Line Up Order:
1: Reese Atwood
2: Katie Stewart
3: Kayden Henry
4: Leighann Goode
5: Joley Mitchell
6: Mia Scott
7: Vanessa Quiroga
8: Ashton Maloney
9: Victoria Hunter
Average runs per game = 7.07


# Next Steps
- take into account opposing team pitching / defense
- take into account home or away
- simulate steals, douple plays, errors, etc.